In [ ]:
"""Evaluate single-physics and multiphysics posteriors for two SNRs."""

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc
from pathlib import Path

import mpebia.plotting.rc_params  # pylint: disable=unused-import
from mpebia.electromechanical_model.cube_model import CubeModel
from mpebia.electromechanical_model.parameters_shared import ParametersShared
from mpebia.entropies import (
    entropy_2d,
    information_gain_2d,
    relative_increase_in_information_gain
)
from mpebia.logging import get_logger
from mpebia.observations import get_equidistant_indices, sample_sobol, snr_to_std
from mpebia.output import get_object_attributes
from mpebia.plotting import colors
from mpebia.plotting.positioning import get_axis_bbox
from mpebia.truncated_gaussian_prior import TruncatedNormalPrior

# SETUP

logger = get_logger("./evaluate_and_plot_posteriors.log")
directory = Path(".")

params = ParametersShared()
cube_model = CubeModel(params.l0, params.U, params.rho)
prior = TruncatedNormalPrior(
    params.mean_prior,
    params.std_prior,
    params.truncations_E,
    params.truncations_nu,
    (params.num_grid_points_E, params.num_grid_points_nu),
    params.offset_ppf,
)

logger.info("Parameters shared:\n%s", get_object_attributes(params))


# COMPUTATIONS

# Compute prior
log_prior_grid = prior.get_log_prior_on_grid()
prior_grid = np.exp(log_prior_grid)
prior.plot_prior_and_grid_points(
    prior_grid,
    [params.E_gt, params.nu_gt],
    directory,
    scale_dim1=1 / 1000,
)

# Compute displacement observations with ground truth values
cube_model.set_material_parameters(params.E_gt, params.nu_gt)
Fs = np.linspace(0.0, params.F_max, np.max(params.nums_obs_I) + 2)
d_gt, I_gt = cube_model.solve(Fs)

max_i_obs = len(Fs)
i_obs_d = get_equidistant_indices(params.num_obs_d, max_i_obs)
F_obs_d = Fs[i_obs_d]
d_obs_wo_noise, _ = cube_model.solve(F_obs_d)

std_d = snr_to_std(params.snr_d, d_obs_wo_noise)
logger.info("Standard deviation d: %f\n", std_d)

# Sample noise for d from a Sobol sequence
noise = sample_sobol(len(d_obs_wo_noise), params.seed)
d_obs = d_obs_wo_noise + noise * std_d

# Compute displacement log-likelihood on the parameter grid
log_likelihood_grid_d = cube_model.get_log_likelihood_on_grid(
    prior, d_obs, std_d, F_obs_d, index_sol=0
)

# Compute single-physics posterior, information gain, and entropy
info_gain_sp, posterior_sp = information_gain_2d(
    log_likelihood_grid_d,
    log_prior_grid,
    prior.grid_points_1,
    prior.grid_points_2,
    return_posterior=True,
)
entropy_post_sp = entropy_2d(posterior_sp, prior.grid_points_1, prior.grid_points_2)
logger.info("######## Single-physics BIA ########")
logger.info("Entropy of single-physics posterior: %f\n", entropy_post_sp)
logger.info("Information gain single-physics: %f", info_gain_sp)
logger.info("####################################\n")

info_gains = [info_gain_sp]
riigs = []
posterior_grids = [posterior_sp]
Fs_obs_I = []
Is_obs = []

for num_obs_I, snr_I in zip(params.nums_obs_I, params.snrs_I):
    # Compute electric current observations with ground truth values
    i_obs_I = get_equidistant_indices(num_obs_I, max_i_obs)
    F_obs_I = Fs[i_obs_I]
    Fs_obs_I.append(F_obs_I)
    cube_model.set_material_parameters(params.E_gt, params.nu_gt)
    _, I_obs_wo_noise = cube_model.solve(F_obs_I)

    std_I = snr_to_std(snr_I, I_obs_wo_noise)
    logger.info("Standard deviation I: %f\n", std_I)
    noise = sample_sobol(len(I_obs_wo_noise), params.seed)
    I_obs = I_obs_wo_noise + noise * std_I
    Is_obs.append(I_obs)

    # Compute electric current log-likelihood on the parameter grid
    log_likelihood_grid_I = cube_model.get_log_likelihood_on_grid(
        prior, I_obs, std_I, F_obs_I, index_sol=1
    )
    log_likelihood_grid_both = log_likelihood_grid_d + log_likelihood_grid_I

    # Compute posterior, information gain, and entropy
    info_gain_mp, posterior_mp = information_gain_2d(
        log_likelihood_grid_both,
        log_prior_grid,
        prior.grid_points_1,
        prior.grid_points_2,
        return_posterior=True,
    )

    riig = relative_increase_in_information_gain(info_gain_sp, info_gain_mp) 
    logger.info("######## Multiphysics BIA with N_obs=%d, SNR=%f ########", num_obs_I, snr_I)
    logger.info("Information gain multiphysics: %f", info_gain_mp)
    logger.info("RIIG = (IG_mp - IG_sp) / IG_sp: %f", riig)
    logger.info("##################################################\n")

    info_gains.append(info_gain_mp)
    riigs.append(riig)
    posterior_grids.append(posterior_mp)

In [ ]:
# PLOTTING

# General plot parameters
plt.rcParams["hatch.color"] = "grey"
plt.rcParams["hatch.linewidth"] = 3.0
num_rows = 5
num_columns = 3
row_contours = 2
row_ig = 3
row_riig = 4
height_ratios = [4, 4, 5.6, 0.9, 0.9]
fig, ax = plt.subplots(
    num_rows,
    num_columns,
    gridspec_kw={"height_ratios": height_ratios, "wspace": 0.45, "hspace": 0.55},
    figsize=(11.9, 11.0),
)
fig.subplots_adjust(left=0.11, right=0.83, top=0.85, bottom=0.07)


################## Plot numbers ##################
pos = np.array([0.5, 1.17])
for col in [1, 2]:
    ax[0, col].scatter(
        pos[0],
        pos[1],
        color="k",
        s=250,
        marker="o",
        zorder=10,
        transform=ax[0, col].transAxes,
        clip_on=False,
    )
    ax[0, col].scatter(
        pos[0],
        pos[1],
        color="w",
        s=200,
        marker="o",
        zorder=11,
        transform=ax[0, col].transAxes,
        clip_on=False,
    )
    ax[0, col].annotate(
        r"$\bf{" + str(col) + r"}$",
        pos,
        xytext=pos + (0.005, -0.007),
        color="k",
        fontsize=14,
        ha="center",
        va="center",
        zorder=12,
        xycoords=ax[0, col].transAxes,
        textcoords=ax[0, col].transAxes,
    )

################## Plot observations ##################
ax[1, 0].axis("off")
for col in range(num_columns):
    ax[0, col].plot(Fs, d_gt * 1000, label="Ground truth", color=colors.GROUND_TRUTH)
    ax[0, col].scatter(
        F_obs_d,
        d_obs * 1000,
        label="Observations",
        color=colors.OBSERVATIONS_DARK,
        s=12,
    )
    ax[0, col].set_ylabel(r"$y_1 = d$ [mm]")
for col in [1, 2]:
    ax[1, col].plot(Fs, I_gt * 1000, label="Ground truth", color=colors.GROUND_TRUTH)
    ax[1, col].scatter(
        Fs_obs_I[col - 1],
        Is_obs[col - 1] * 1000,
        label="Observations",
        color=colors.OBSERVATIONS_DARK,
        s=12,
    )
    ax[1, col].set_ylabel(r"$y_2 = I$ [mA]")
    ax[1, col].set_ylim(np.min(Is_obs[1]) * 1000 - 3, np.max(Is_obs[1]) * 1000 + 3)
for row in range(2):
    for col in range(num_columns):
        ax[row, col].set_xlabel(r"$F$ [N]")
        ax[row, col].grid(True)
# Add legend
ax[0, 2].legend(bbox_to_anchor=(1.15, -0.45), loc="lower left")

################## Plot posteriors ##################
for col, posterior in zip(range(num_columns), posterior_grids):
    contour = ax[row_contours, col].contourf(
        prior.grid_points_1 / 1000,
        prior.grid_points_2,
        posterior.T,
        6,
        vmin=np.min(posterior_grids),
        vmax=np.max(posterior_grids),
        cmap=colors.CMAP,
    )
    ax[row_contours, col].scatter(
        params.E_gt / 1000,
        params.nu_gt,
        marker="x",
        color=colors.GROUND_TRUTH,
        linewidths=1.5,
        s=100,
        label="Ground truth",
        zorder=3,
    )
    ax[row_contours, col].scatter(
        params.E_gt / 1000,
        params.nu_gt,
        marker="x",
        color="white",
        linewidths=3.0,
        s=120,
    )
    ax[row_contours, col].set_xlabel(r"$x_1 = E$ [kPa]")
    ax[row_contours, col].set_ylabel(r"$x_2 = \nu$ [\hspace{1pt}-\hspace{1pt}]")
    ax[row_contours, col].grid(True)
    ax[row_contours, col].set_xlim(9, 13)
    ax[row_contours, col].set_ylim(0.15, np.max(prior.grid_points_2))
# Add colorbar
points_last_contour = ax[row_contours, 2].get_position().get_points()
height_last_contour = points_last_contour[1, 1] - points_last_contour[0, 1]
cbar_ax = fig.add_axes(
    [
        points_last_contour[1, 0] + 0.035,
        points_last_contour[0, 1] + 0.035,
        0.03,
        height_last_contour - 0.035,
    ]
)
cbar = fig.colorbar(contour, cax=cbar_ax)
cbar.set_label(r"Posterior $p(\boldsymbol{x} | \boldsymbol{y}_\text{obs})$")
# Add legend
ax[row_contours, 2].legend(bbox_to_anchor=(1.15, -0.14), loc="lower left")

################## Plot IG ##################
ig_upper_lim = np.max(info_gains) * 1.05
for col, info_gain_mp in zip(range(num_columns), info_gains):
    if col in [1, 2]:
        # Dummy bar for legend
        ax[row_ig, col].barh(
            0.0,
            0.0,
            color="grey",
            label="Single-physics",
        )
        # Actual bars
        ax[row_ig, col].barh(
            0.0,
            info_gain_mp,
            color=colors.MULTIPHYSICS,
            label="Multi-physics",
        )
        ax[row_ig, col].barh(0.0, info_gains[0], color="none", hatch="//")
        ax[row_ig, col].vlines(info_gains[0], -0.4, 0.4, color="grey")
        ax[row_ig, col].set_xlabel(
            r"$\text{IG}\!\left( \boldsymbol{y}_\text{obs} \right)$ [nat]", labelpad=4.0
        ) 
    else:
        ax[row_ig, col].barh(0.0, info_gain_mp, color="grey")        
        ax[row_ig, col].set_xlabel(
            r"$\text{IG}\!\left( \boldsymbol{y}_\text{obs} = \boldsymbol{y}_{1, \text{obs}} \right)$ [nat]", labelpad=4.0
        ) 
    ax[row_ig, col].set_xlim(0.0, ig_upper_lim)
    ax[row_ig, col].set_ylim(-0.6, 0.6)
    ax[row_ig, col].get_yaxis().set_visible(False)
    ax[row_ig, col].spines["top"].set_visible(False)
    ax[row_ig, col].spines["right"].set_visible(False)
    ax[row_ig, col].grid(True)
# Add IG legend
ax[row_ig, 2].legend(bbox_to_anchor=(1.15, 0.25), loc="center left")

################## Plot RIIG ##################
ax[row_riig, 0].axis("off")

for i, riig in enumerate(riigs):
    col = i + 1
    box = ax[row_riig, col].get_position()
    left_offset = info_gain_sp / ig_upper_lim
    ax[row_riig, col].set_position([
        box.x0 + box.width * left_offset,  # move it right
        box.y0,
        box.width * (1-left_offset),  # make it narrower
        box.height
    ])
    ax[row_riig, col].barh(
        0.0,
        riig,
        color=colors.MULTIPHYSICS,
    )
    logger.info("RIIG for col %d: %f", col, riig)

    ax[row_riig, col].set_xlim(0.0, riig * (1.0 + 0.05 / (1-left_offset)))
    ax[row_riig, col].set_ylim(-0.6, 0.6)
    xlabel = r"$\text{RIIG}\!\left( \boldsymbol{y}_{1, \text{obs}}, \boldsymbol{y}_{2, \text{obs}} \right)$ [\hspace{1pt}-\hspace{1pt}]"
    ax[row_riig, col].set_xlabel(xlabel, labelpad=4.0, loc="right")
    ax[row_riig, col].get_yaxis().set_visible(False)
    ax[row_riig, col].spines["top"].set_visible(False)
    ax[row_riig, col].spines["right"].set_visible(False)
    ax[row_riig, col].grid(True)

################## Add colored boxes ##################
gap = 0.01
pad = 0.01
extra_pad = 0.01
height_text = 0.03
height_boxes = 1 - 2 * pad - height_text
width_legend = 0.18
width_labels = 0.04
linewidth = 5
label_sp = r"\textbf{Single-physics BIA}"
label_mp = r"\textbf{Multi-physics BIA}"
labels = [label_sp, label_mp]

# Get column positions
pos_col1 = get_axis_bbox(ax[2, 0], fig)
pos_col2 = get_axis_bbox(ax[1, 1], fig)
pos_col3 = ax[2, 2].get_position()
x_center_col12 = (pos_col1.x1 + pos_col2.x0) / 2
positions = [
    (pos_col1.x0 - pad, pad),
    (x_center_col12 + gap / 2, pad),
]
width_left_box = x_center_col12 - positions[0][0] - gap / 2
width_right_box = pos_col3.x1 - positions[1][0] + 1.5 * pad
widths = [width_left_box, width_right_box]
colors_frame = [colors.MODEL, colors.MULTIPHYSICS_LIGHT]
zorders = [-1000, -1000]
zipped = zip(labels, positions, widths, colors_frame, zorders)
for label, pos, width, color, zorder in zipped:
    # frame
    fig.patches.append(
        plt.Rectangle(
            pos,
            width,
            height_boxes,
            fill=False,
            edgecolor=color,
            linewidth=linewidth,
            transform=fig.transFigure,
            figure=fig,
            zorder=zorder,
        )
    )
    if label == label_mp:
        # Additional box inbetween single-physics axes
        fig.patches.append(
            plt.Rectangle(
                pos,
                width / 2,
                height_boxes,
                fill=False,
                edgecolor=color,
                linewidth=linewidth,
                transform=fig.transFigure,
                figure=fig,
                zorder=zorder,
            )
        )
    # background for label
    fig.patches.append(
        plt.Rectangle(
            (pos[0], pos[1] + height_boxes),
            width,
            height_text,
            fill=True,
            facecolor=color,
            edgecolor=color,
            linewidth=linewidth,
            transform=fig.transFigure,
            figure=fig,
            zorder=zorder,
        )
    )
    # label
    fig.text(
        pos[0] + (width / 2),
        pos[1] + height_boxes + height_text / 2,
        label,
        verticalalignment="center",
        horizontalalignment="center",
        fontsize=16,
        weight="bold",
    )

##### Add row labels #####
pos_row1 = ax[0, 0].get_position()
pos_row2 = ax[1, 0].get_position()
pos_row3 = ax[2, 0].get_position()
pos_row4 = ax[3, 0].get_position()
pos_row5 = ax[4, 0].get_position()
# Calculate the y-coordinate that is centered between the first two rows
y_center_row12 = (pos_row1.y0 + pos_row2.y1) / 2
y_center_row1 = (pos_row1.y0 + pos_row1.y1) / 2
y_center_row2 = (pos_row2.y0 + pos_row2.y1) / 2
y_center_row3 = (pos_row3.y0 + pos_row3.y1) / 2
y_center_row4 = (pos_row4.y0 + pos_row4.y1) / 2
y_center_row5 = (pos_row5.y0 + pos_row5.y1) / 2 - 0.02
# Loop over labels
positions_x = [0.02] * 5
positions_y = [y_center_row1, y_center_row2, y_center_row3, y_center_row4, y_center_row5]
labels = [
    "First-field\nobservations",
    "Second-field\nobservations",
    "Posterior",
    "Information\ngain",
    "RIIG",
]
for pos_x, pos_y, label in zip(positions_x, positions_y, labels):
    fig.text(
        pos_x,  # x position
        pos_y,  # y position
        label,
        rotation="vertical",
        verticalalignment="center",
        horizontalalignment="center",
        fontsize=16,
        weight="bold",
    )

##### Add MP column labels #####
# Calculate the centered x-coordinate of each column
x_center_col2 = x_center_col12 + gap / 2 + width_right_box / 4
x_center_col3 = x_center_col12 + gap / 2 + width_right_box * 3 / 4
y_center = get_axis_bbox(ax[0, 1], fig).y1 + 0.056
# Loop over labels
positions_x = [x_center_col2, x_center_col3]
labels = [
    r"Low number of observations $N_{\text{obs}, 2}$",
    r"Low signal-to-noise ratio $\text{SNR}_2$",
]
for pos_x, label in zip(positions_x, labels):
    labels_per_line = [label, "in the second field"]
    for label_i, offset in zip(labels_per_line, [0, -0.025]):
        fig.text(
            pos_x,  # x position
            y_center + offset,  # y position
            label_i,
            verticalalignment="center",
            horizontalalignment="center",
            fontsize=14,
            weight="bold",
        )
    fig.patches.append(
        plt.Rectangle(
            (pos_x - width_right_box / 4, y_center - 0.04),
            width_right_box / 2,
            0.06,
            fill=True,
            facecolor=colors.MULTIPHYSICS_LIGHTER,
            edgecolor=None,
            linewidth=linewidth,
            transform=fig.transFigure,
            figure=fig,
            zorder=zorder - 1,
        )
    )

plt.savefig(
    directory / "posteriors_with_observations.png",
    dpi=300,
)


In [ ]:
info_gain_mp, posterior_mp = information_gain_2d(
    log_likelihood_grid_I,
    log_prior_grid,
    prior.grid_points_1,
    prior.grid_points_2,
    return_posterior=True,
)

plt.figure(figsize=(4, 4))
plt.contourf(
        prior.grid_points_1 / 1000,
        prior.grid_points_2,
        posterior_mp.T,
        6,
        vmin=np.min(posterior_grids),
        vmax=np.max(posterior_grids),
        cmap=colors.CMAP,
    )
plt.xlabel(r"$E$")
plt.ylabel(r"$\nu$")
plt.grid(True)
plt.tight_layout()
plt.xlim(9, 13)
plt.ylim(0.15, np.max(prior.grid_points_2))
plt.show()

info_gain_mp

In [ ]:
info_gain_mp, posterior_mp = information_gain_2d(
    log_likelihood_grid_both,
    log_prior_grid,
    prior.grid_points_1,
    prior.grid_points_2,
    return_posterior=True,
)

plt.figure(figsize=(4, 4))
plt.contourf(
        prior.grid_points_1 / 1000,
        prior.grid_points_2,
        posterior_mp.T,
        6,
        vmin=np.min(posterior_grids),
        vmax=np.max(posterior_grids),
        cmap=colors.CMAP,
    )
plt.xlabel(r"$E$")
plt.ylabel(r"$\nu$")
plt.grid(True)
plt.tight_layout()
plt.xlim(9, 13)
plt.ylim(0.15, np.max(prior.grid_points_2))
plt.show()

info_gain_mp